In [ ]:
# Import key functions from package, also helps to have ase and other standard packages, such as numpy.
import ase
import numpy as np

from atomaton.analyze_structure import guess_bonds
from atomaton.visualize import view_structure

In [ ]:
# Calculating bonds between atoms.
"""
Many types of structure files do not have bond information, which means that these needs to be calculated to be used with certain forcefields.
To calculate bonds with atomaton, we first define a dictionary of bond lengths (in angstrom) for various atom type pairs.
Note that atom types should be listed in alphabetical order so that the default bond length is not used.
(ex. S-C will be skipped, because the code will always order the atoms types as C-S when checking the dictionary.)

TODO: Add bond dictionary validation code.
"""

bond_dict = {'default':[0,2.0], 'C-C':[0, 1.8], 'C-Cl':[0, 1.8], 'C-S':[0, 1.9], 'O-S':[0, 1.9]}

ethane = ase.io.read("ethane.xyz")
irmof1 = ase.io.read("IRMOF-1.cif")

# TODO: Refactor this function, it's a mess and returns too much junk.
_, all_bonds, _, _, _, extra_atoms_for_plot, extra_bonds_for_plot = guess_bonds(irmof1, np.zeros(len(irmof1)), cutoff=bond_dict, periodic=None)

In [19]:
# Viewing structures

# Set view parameters
figure = {'bgcolor':(1,1,1), 'size':(800,800)}
objects = {'cell_r':0.15}
camera = {'azimuth': 90, 'elevation': 90, 'distance': None, 'parallel': False}

# View the structure
# By default, this is not ineractive, and does not save an image, which means it will render an image and then close it. 
# If you want to view it interactively, use the kwarg, or use the filename kwarg to save the image.
view_structure(irmof1, all_bonds, [], show_unit_cell=True, figure=figure, objects=objects, camera=camera, filename=None, interactive=True)